In [1]:
import torch
import torch.nn as nn

In [ ]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "emb_dim": 768,          # Embedding dimension
    "n_heads": 12,           # Number of attention heads
    "n_layers": 12,          # Number of layers
    "drop_rate": 0.1,        # Dropout rate
    "qkv_bias": False        # Query-Key-Value bias
}

In [3]:
class DummyTransformerBlog(nn.Module):
    def __init__(self, cfg):
        super().__init__()

In [4]:
class DummyLayerNorm(nn.Module):
    def __init__(self, emb_dim, eps=1e-6):
        super().__init__()
        self.eps = eps

In [5]:
class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg

        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        #Shouldn't there be the Positional Embedding implemented?
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trns_blocks = nn.Sequential(*[DummyTransformerBlog(cfg) for _ in range(cfg["n_layers"])])
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias = False)

    def forward(self, x_in):
        batch_size, seq_length = x_in.shape
        x = x_in
        tok_embeds  = self.tok_emb(x_in)
        pos_emb = self.pos_emb(torch.arange(seq_length, device=x_in.device))
        x = tok_embeds + pos_emb
        x = self.drop_emb(x)

        x = self.trns_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)

        return logits



